In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from solcore.structure import Junction
from solcore.solar_cell import SolarCell
from solcore.light_source import LightSource
from solcore.spice import solve_quasi_3D
config.smarts(r'/home/jovyan/smarts')
config.spice(r'/home/jovyan/S4')

First we load the masks defining the illumination pattern and the contacts. Both must be greyscale images<br>
The solver expect images with values between 0 and 255 and imread of a PNG image is between 0 and 1, even when<br>
it is in grey, so we scale it multiplying by 255. If the image were JPG, the result would be already in (0,255).

In [3]:
illuminationMask = (plt.imread('../data/masks_illumination.png') * 255).astype(np.int)
contactsMask = (plt.imread('../data/masks_sq.png') * 255).astype(np.int)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your c

In [4]:
nx, ny = illuminationMask.shape

For symmetry arguments (not completely true for the illumination), we can mode just 1/4 of the device and then<br>
multiply the current by 4

In [5]:
illuminationMask = illuminationMask[int(nx / 2):, int(ny / 2):]
contactsMask = contactsMask[int(nx / 2):, int(ny / 2):]

Size of the pixels (m)

In [6]:
Lx = 10e-6
Ly = 10e-6

Height of the metal fingers (m)

In [7]:
h = 2.2e-6

Contact resistance (Ohm m2)

In [8]:
Rcontact = 3e-10

Resistivity metal fingers (Ohm m)

In [9]:
Rline = 2e-8

Bias (V)

In [10]:
vini = 0
vfin = 1.3
step = 0.01

In [11]:
T = 298
db_junction = Junction(kind='2D', T=T, reff=1, jref=300, Eg=0.66, A=1, R_sheet_top=100, R_sheet_bot=1e-16,
                       R_shunt=1e16, n=3.5)
db_junction2 = Junction(kind='2D', T=T, reff=1, jref=300, Eg=1.4, A=1, R_sheet_top=100, R_sheet_bot=1e-16,
                        R_shunt=1e16, n=3.5)
db_junction3 = Junction(kind='2D', T=T, reff=0.5, jref=300, Eg=1.8, A=1, R_sheet_top=100, R_sheet_bot=100,
                        R_shunt=1e16, n=3.5)

For a single junction, this will have >28800 nodes and for the full 3J it will be >86400, so it is worth to<br>
exploit symmetries whenever possible. A smaller number of nodes also makes the solver more robust.

In [12]:
my_solar_cell = SolarCell([db_junction2], T=T)

In [13]:
wl = np.linspace(350, 2000, 301) * 1e-9
light_source = LightSource(source_type='standard', version='AM1.5g', x=wl, output_units='photon_flux_per_m',
                           concentration=100)

In [14]:
options = {'light_iv': True, 'wavelength': wl, 'light_source': light_source, 'optics_method': 'BL'}

In [15]:
V, I, Vall, Vmet = solve_quasi_3D(my_solar_cell, illuminationMask, contactsMask, options=options, Lx=Lx, Ly=Ly, h=h,
                                  R_back=1e-16, R_contact=Rcontact, R_line=Rline, bias_start=vini, bias_end=vfin,
                                  bias_step=step)

Solving 1D Solar Cell...
Solving optics of the solar cell...
Solving IV of the junctions...
Solving IV of the tunnel junctions...
Solving IV of the total solar cell...
... Done!

Solving quasi-3D Solar Cell...


SpiceSolverError: Spice solver not found.

Since we model 1/4 of the device, we multiply the current by 4

In [16]:
import os
from solcore import config
print(os.getcwd())


/home/jovyan/examples/notebooks
<bound method SolcoreConfig.spice of [Configuration]
version = 5.7.2
welcome_message = 1
verbose_loading = 1

[Units]
default = SOLCORE_ROOT/units_system/Default_units.txt

[Parameters]
calculables = SOLCORE_ROOT/parameter_system/calculable_parameters.txt
adachi_binaries = SOLCORE_ROOT/material_data/Adachi/binaries.txt
adachi_compounds = SOLCORE_ROOT/material_data/Adachi/ternaries.txt
vurgaftman_binaries = SOLCORE_ROOT/material_data/Vurgaftman/binaries.txt
vurgaftman_compounds = SOLCORE_ROOT/material_data/Vurgaftman/ternaries.txt
levinshtein_groupIV = SOLCORE_ROOT/material_data/Levinshtein/GroupIV.txt

[Materials]
GaAs = SOLCORE_ROOT/material_data/GaAs-Material
InSb = SOLCORE_ROOT/material_data/InSb-Material
GaSb = SOLCORE_ROOT/material_data/GaSb-Material
AlAs = SOLCORE_ROOT/material_data/AlAs-Material
AlGaAs = SOLCORE_ROOT/material_data/AlGaAs-Material
InGaAs = SOLCORE_ROOT/material_data/InGaAs-Material
GaInSb = SOLCORE_ROOT/material_data/InGaSb-Materia

In [ ]:
I = I * 4

In [ ]:
plt.figure(1)
plt.imshow(Vall[:, :, -2, -1])

plt.figure(2)
plt.semilogy(V, abs(I))
plt.show()